In [ ]:
import numpy as np
import pandas as pd
from win32com.client import Dispatch
import time
Server = Dispatch("PX32.OpenServer.1")
#Server.DoCommand('GAP.START()')
path = r'C:\\1 Work\\Новая папка\\OpSer_1.txt'

df = pd.read_csv(path, sep = '\t')
well = df['Well'].values.tolist()
date = df['Dates'].values.tolist()
res_pres = df['Res_pres'].values.tolist()
wct = df['WCT'].values.tolist()
gor = df['GOR'].values.tolist()
pi = df['PI'].values.tolist()


def dates_predict (date):

    year=date[6:]
    month=date[3:5]
    month_step=str(int(month)+1)
    day=date[:2]
    if month=='12':
        month_step='01'
        year=str(int(year)+1)
    date_ready=str(day)+'.'+str(month)+'.'+str(int(year)+1)
    return date_ready


def Start_pred(Server, date):
    Server.DoCommand('GAP.PREDINIT()')
    #Server.DoCommand('GAP.PredDoSolver(0,0,0,{PROD}')
    Server.DoCommand('GAP.PredDoStep()')
    Server.DoCommand('GAP.PREDEND()')

for i in range(len(well)):
    well[i]=str(well[i])
    Server.SetValue('GAP.MOD[{PROD}].WELL[{%s}].IPR[0].WCT' %(str(well[i])), str(wct[i]))
    Server.SetValue('GAP.MOD[{PROD}].WELL[{%s}].IPR[0].ResPres' %(well[i]), str(res_pres[i]))
    Server.SetValue('GAP.MOD[{PROD}].WELL[{%s}].IPR[0].GOR' %(well[i]), str(gor[i]))
    Server.SetValue('GAP.MOD[{PROD}].WELL[{%s}].IPR[0].PI' % (well[i]), str(pi[i]))

    Server.SetValue('GAP.MOD[{PROD}].PREDINFO.START', '{'+str(date[0])+'}')
    Server.SetValue('GAP.MOD[{PROD}].PREDINFO.END', '{'+dates_predict(str(date[0]))+'}')
    Server.SetValue('GAP.MOD[{PROD}].PREDINFO.STEPTYPE', 'months')
    Server.SetValue('GAP.MOD[{PROD}].PREDINFO.STEP', '1')

t1 = time.time()
Start_pred(Server, date[i])
t2 = time.time()

print(t2-t1)

Server = None
